## LITTLE LEMON ANALYSIS AND SALES REPORT

In [1]:
#Install connector api using the command below.
!pip install mysql-connector-python
print("mysql-connector is now intalled")

mysql-connector is now intalled


In [3]:
#Checking if mysql-connector is installed
!pip show mysql-connector
print("mysql-connector is present")

Name: mysql-connector
Version: 2.2.9
Summary: MySQL driver written in Python
Home-page: http://dev.mysql.com/doc/connector-python/en/index.html
Author: Oracle and/or its affiliates
Author-email: 
License: GNU GPLv2 (with FOSS License Exception)
Location: c:\users\violet\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: 
mysql-connector is present


#### Task 1: Create a connection pool
Required Steps:
* Import MySQLConnectionPool class from MySQL Connector/Python.
* Import the Error class from MySQL Connector/Python.To find the information on the error
* Define your database configurations as a Python dictionary object called dbconfig.
* Establish a connection pool [pool_name = pool_b] with two connections. 
* Implement error handling using a try-except block in case the connection fails. 

In [6]:
#TASK 1: CREATING A CONNECTION POOL

#import MySQLConnectionPool  & Error class from MySQL Connector/Python.
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error 

#Defining my database configurations as a Python dictionary object called dbconfig.
dbconfig={"database":"little_lemon_db", "user":"violet", "password":"greenin2023", "host":"localhost", 
    "port" : 3306, "auth_plugin":"mysql_native_password"}

#Establishing a connection pool [pool_name = pool_b] with two connections while 
#implementing error handling using a try-except block in case the connection fails
try:
    pool2=MySQLConnectionPool(pool_name="pool_b",pool_size=2, **dbconfig)
    print("The connection pool is created with a name: ",pool2.pool_name)
    print("The pool size is: ",pool2.pool_size)
    
except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_b
The pool size is:  2


#### Task 2: Connecting guests to the pool
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the data in the Bookings table
* Guest 1:Table Number: 8, First Name: Anees, Last Name: Java, Booking Time: 18:00, EmployeeID: 6
* Guest 2:Table Number: 5, First Name: Bald, Last Name: Vin, Booking Time: 19:00, EmployeeID: 6
* Guest 3:Table Number: 12, First Name: Jay, Last Name: Kon, Booking Time: 19:30, EmployeeID: 6

Return all the connections back to the pool. 

In [7]:
# TASK 2: GET CONNECTIONS TO THE POOL FOR 3 GUESTS TRYING TO BOOK SIMULTANEOUSLY

# Step 1: Getting connection for first guest 1
guests = ["Anees Java", "Bald Vin", "Jay Kon"]
connection1 = pool2.get_connection()
print ("A guest {} with connection ID {} is requsting to add a booking to the database.".format(guests[0], 
connection1.connection_id))

#creating cursor object
cursor = connection1.cursor()
print("cursor object created")

#query to insert guest 1's booking
guest1_query = """INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)values (8, "Anees", "Java","18:00:00", 6);"""

#executing guest 1's query
cursor.execute(guest1_query)
print("Booking is for {} is created.\n" .format(guests[0]))

A guest Anees Java with connection ID 387 is requsting to add a booking to the database.
cursor object created
Booking is for Anees Java is created.



In [8]:
#Step 2: Getting connection for guest 2
guests = ["Anees Java", "Bald Vin", "Jay Kon"]
connection2 = pool2.get_connection()
print ("A guest {} with connection ID {} is requsting to add a booking to the database.".format(guests[0+1], 
connection2.connection_id))

#creating cursor object
cursor = connection2.cursor()
print("cursor object created")

#query to insert guest 2's booking
guest2_query = """INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)values (5, "Bald", "Vin", "19:00:00",6);"""

#executing guest 1's query
cursor.execute(guest2_query)
print("Booking is for {} is created.\n" .format(guests[0+1]))


A guest Bald Vin with connection ID 388 is requsting to add a booking to the database.
cursor object created
Booking is for Bald Vin is created.



In [9]:
#Step 3: Trying to connect a third user after the first two will throw an error, I make an instance of MySQLConnection 
#in the next cell below
guests = ["Anees Java", "Bald Vin", "Jay Kon"]
connection3 = pool2.get_connection()
print ("A guest {} with connection ID {} is requsting to add a booking to the database."
.format(guests[0+2], connection3.connection_id))

#creating cursor object
cursor = connection3.cursor()
print("cursor object created")

#query to insert guest 3's booking
guest3_query = """INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)values (12, "Jay", "Kon", "19:30:00", 6);"""

#executing guest 3's query
cursor.execute(guest3_query)
print("Booking is for {} is created.\n" .format(guests[0+2]))

PoolError: Failed getting connection; pool exhausted

In [10]:
# Step 4:  Creating an instance of MySQLConnection to add an extra connection to the pool for guest 3

#importing connector 
import mysql.connector as connector
guests = ["Anees Java", "Bald Vin", "Jay Kon"]

#create a connection
connection=connector.connect(user="violet",password="greenin2023",host="localhost", 
    port=3306, auth_plugin ="mysql_native_password")

#add the connection into the pool
pool2.add_connection(cnx=connection)
print("A third connection is added in the pool")
        
connection3guest = pool2.get_connection()
print("New connection: {} can now add a booking to the database.".format(guests[0+2])) 

#query to insert guest 3's booking
guest3_query = """INSERT INTO Bookings (TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)values (12, "Jay", "Kon", "19:30:00", 6);"""

#executing guest 3's query
cursor.execute(guest3_query)
print("Booking is for {} is created.\n" .format(guests[0+2]))

A third connection is added in the pool
New connection: Jay Kon can now add a booking to the database.
Booking is for Jay Kon is created.



In [11]:
#Step 5: Closing the all 3 connections using try-except block to print the error message

try:
        connection1.close()
        print("Connection 1 is closed")
        connection2.close()
        print("Connection 2 is closed")
        connection3guest.close()
        print("Guest connection 3 is closed")
except:
        print("Extra connection/s not  yet closed")
 

Connection 1 is closed
Connection 2 is closed
Extra connection/s not  yet closed


#### Task 3: Create a report
Create a report containing the following information:
* The name and EmployeeID of the Little Lemon manager.
* The name and role of the employee who receives the highest salary.
* The number of guests booked between 18:00 and 20:00.
* The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [12]:
#TASK 3: CREATE A REPORT

#getting a connection from the pool
connection1 = pool2.get_connection()
print("You got connected to", pool2.pool_name, "with connection ID", connection1.connection_id)

#create a cursor.
cursor = connection1.cursor()
print("cursor created")

#query to find manager and their EmployeeID
report_query1 = """select Name, EmployeeID from Employees where Role = "Manager";"""
#query to find highest paid Employee
report_query2 = """select Name, Role, Annual_Salary from Employees order by Annual_Salary desc limit 1;"""
#query to find number of guests booked at specific time interval
report_query3 = """select count(BookingID) from Bookings where BookingSlot between "18:00" and "20:00";"""
#query to find guests waiting to be seated
report_query4 = """select BookingID, concat(GuestfirstName,' ', GuestLastName) as "Guest Name" from Bookings 
order by BookingSlot asc;"""
print("LITTLE LEMON REPORT")

#executing report_query1
cursor.execute(report_query1)
#getting query1 results
results=cursor.fetchall()
#displaying query1 results
for result in results:
    print("The manager is ", result [0], "with an employee ID of: ", result[1])

#executing report_query2
cursor.execute(report_query2)
#getting query2 results
results=cursor.fetchall()
#displaying query2 results
for result in results:
    #print (results)
    print("The highest paid employee is:", result [0]," Their role is:", result[1]," At an annual salary of:", result[2])

#executing report_query3
cursor.execute(report_query3)
#getting query3 results
results=cursor.fetchall()
#displaying query3 results
for result in results:
    print("The number of guests booked between 18:00 and 20:00 is ", result[0]) 

#executing report_query4
cursor.execute(report_query4)
#getting query4 results
results=cursor.fetchall()
#displaying query4 results
print("The guests waiting to be seated are:")
for result in results:
    print(result)

#closing connection
connection1.close()
print("Connection1 is closed")


You got connected to pool_b with connection ID 387
cursor created
LITTLE LEMON REPORT
The manager is  Mario Gollini with an employee ID of:  1
The highest paid employee is: Mario Gollini  Their role is: Manager  At an annual salary of: $70,000
The number of guests booked between 18:00 and 20:00 is  4
The guests waiting to be seated are:
(3, 'Vanessa McCarthy')
(4, 'Marcos Romero')
(5, 'Hiroki Yamane')
(1, 'Anna Iversen')
(2, 'Joakim Iversen')
(6, 'Diana Pinto')
Connection1 is closed


Task 4: Create a stored procedure

Create a stored procedure named BasicSalesReport that returns the following statistics: 
* Total sales
* Average sale
* Minimum bill paid
* Maximum bill paid

In [13]:
#TASK 4: CREATING A STORED PROCEDURE NAMED BasicSalesReport

#getting connection from pool
connection2 = pool2.get_connection()
print("You got connection to", pool2.pool_name)
#create a cursor.
cursor = connection2.cursor()
#execute the query
sales_report_query = """
create procedure if not exists BasicSalesReport()
begin
select 
sum(BillAmount) AS TotalSale,
avg(BillAmount) AS AverageSale,
min(BillAmount) AS MinBillPaid,
max(BillAmount) AS MaxBillPaid
from Orders;
end"""

#execute the query
cursor.execute(sales_report_query)

#calling the stored procedure 
cursor.callproc("BasicSalesReport")

#retrieving records 
results = next(cursor.stored_results())
results = results.fetchall()

# Retrieve column names using list comprehension in a for loop
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

print("Basic Sales Report:")
for result in results:
    print(cols[0], ":", result[0])
    print(cols[1], ":", result[1])
    print(cols[2], ":", result[2])
    print(cols[3], ":", result[3])

#closing connection
connection2.close()
print("Connection2 is closed")

You got connection to pool_b
Basic Sales Report:
Total_Sale : 243
Average_Sale : 48.6000
Min_Bill_Paid : 37
Max_Bill_Paid : 86
Connection2 is closed


#### Task 5: Display next 3 upcoming bookings and staff involved
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. 
* Get a connection from the pool.
* Create a buffered cursor.
* Combine the data from the Bookings and the Employee tables. 
* Sort the retrieved records in ascending order. 
* Then display the information of the first three guests. 
* Returned the connection back to the pool.

In [14]:
#TASK 5: DISPLAY THE NEXT 3 UPCOMING BOOKINGS & STAFF ASSIGNED  

#get a connection from the pool created earlier
connection1 = pool2.get_connection()
print("You got connected to", pool2.pool_name, "with connection ID", connection1.connection_id)

#create a buffered cursor.
cursor = connection1.cursor(buffered=True)
print("Buffered cursor created")

#query to retrieve the data from Bookings and Employee tables
upcoming_bookings = """select Hour(Bookings.BookingSlot) as "Booking Slot", concat(Bookings.GuestFirstName,' ',Bookings.GuestLastName) as "GuestName", 
concat(Employees.Name,' ', "[",Employees.Role,"]") as "Assigned to:" from Bookings left join Employees on
Bookings.EmployeeID = Employees.EmployeeID order by hour(BookingSlot);"""
cursor.execute (upcoming_bookings)

#display the information of the first three guests
results= cursor.fetchmany(3)
print("Upcoming Bookings are:")

#displaying reults
for result in results:
    print("[BookingSlot] ", result [0], "[Guest_name] ",result[1], " [Assigned to] ", result[2])

#Returning the connection back to the pool.
connection1.close()
print("Connection1 is closed")

You got connected to pool_b with connection ID 387
Buffered cursor created
Upcoming Bookings are:
[BookingSlot]  15 [Guest_name]  Vanessa McCarthy  [Assigned to]  Giorgos Dioudis [Head Chef]
[BookingSlot]  17 [Guest_name]  Marcos Romero  [Assigned to]  Fatma Kaya [Assistant Chef]
[BookingSlot]  18 [Guest_name]  Hiroki Yamane  [Assigned to]  Adrian Gollini [Assistant Manager]
Connection1 is closed
